In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [20]:
# hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_liters = 250


In [21]:
with open('dataset/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

vocab_size = len(text)
print('vocab_size:', vocab_size)

vocab_size: 232309


In [22]:
string_to_int = {ch:i for i, ch in enumerate(text)}
int_to_string = {i:ch for i, ch in enumerate(text)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda x: ''.join([int_to_string[i] for i in x])
data = torch.tensor(encode(text), dtype=torch.long, device=device)
print('data:', data.shape)

data: torch.Size([232309])


In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


inputs:
tensor([[232235, 232305, 232295, 232294, 232235, 232259, 232275, 232289],
        [232248, 232305, 232287, 232304, 232278, 232304, 232305, 232307],
        [232295, 232307, 232297, 232304, 232278, 232259, 232296, 232275],
        [232284, 232288, 232278, 232308, 232305, 232287, 232288, 232307]],
       device='cuda:0')
targets:
tensor([[232305, 232295, 232294, 232235, 232259, 232275, 232289, 232305],
        [232305, 232287, 232304, 232278, 232304, 232305, 232307, 232259],
        [232307, 232297, 232304, 232278, 232259, 232296, 232275, 231721],
        [232288, 232278, 232308, 232305, 232287, 232288, 232307, 232297]],
       device='cuda:0')


In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

In [26]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = f.softmax(logits, dim=-1) # ( B, C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (8,1)
            # append sampled index to the running sequence
            index = torch.cat((index,  index_next), dim=-1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)


context = torch.zeros((1,1), dtype=torch.long).to(device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 215869885924 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train_loss: {losses['train']:.4f}, val_loss: {losses["val"]: .4f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())